# Pandas tips, tricks and tests

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from   typing      import Tuple, List


## Define a test ntuple

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [4]:
pop = pop.reindex(index)
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### This is a DataFrame with two indexes and two columns. 

In [5]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
state      year                   
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

#### This is a DataFrame with multi-index in rows and columns 

In [6]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      36.0  35.1  48.0  37.9  39.0  38.0
     2      41.0  38.3  41.0  37.7  39.0  36.0
2014 1      30.0  37.5  31.0  37.5  38.0  35.9
     2      32.0  38.6  47.0  36.9  19.0  37.3

## Indexing and Slicing a MultiIndex

Indexing and slicing on a ``MultiIndex`` is designed to be intuitive, and it helps if you think about the indices as added dimensions.
We'll first look at indexing multiply indexed ``Series``, and then multiply-indexed ``DataFrame``s.

### Multiply indexed Series

Consider the multiply indexed ``Series`` of state populations we saw earlier:

In [7]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

We can access single elements by indexing with multiple terms:

In [8]:
pop['California', 2000]

33871648

The ``MultiIndex`` also supports *partial indexing*, or indexing just one of the levels in the index.
The result is another ``Series``, with the lower-level indices maintained:

In [9]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

Partial slicing is available as well, as long as the ``MultiIndex`` is sorted (see discussion in [Sorted and Unsorted Indices](#Sorted-and-unsorted-indices)):

In [10]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:

In [11]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

Other types of indexing and selection (discussed in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb)) work as well; for example, selection based on Boolean masks:

In [12]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

Selection based on fancy indexing also works:

In [13]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

A multiply indexed ``DataFrame`` behaves in a similar manner.
Consider our toy medical ``DataFrame`` from before:

In [14]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      36.0  35.1  48.0  37.9  39.0  38.0
     2      41.0  38.3  41.0  37.7  39.0  36.0
2014 1      30.0  37.5  31.0  37.5  38.0  35.9
     2      32.0  38.6  47.0  36.9  19.0  37.3

Remember that columns are primary in a ``DataFrame``, and the syntax used for multiply indexed ``Series`` applies to the columns.
For example, we can recover Guido's heart rate data with a simple operation:

In [15]:
health_data['Guido', 'HR']

year  visit
2013  1        48.0
      2        41.0
2014  1        31.0
      2        47.0
Name: (Guido, HR), dtype: float64

Also, as with the single-index case, we can use the ``loc``, ``iloc``, and ``ix`` indexers introduced in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb). For example:

In [16]:
health_data.iloc[:1, :1]

,subject,Bob
,type,HR
year,visit,
2013,1,36.0


In [17]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      36.0  35.1
     2      41.0  38.3

In [18]:
health_data.iloc[:3, :3]

subject      Bob       Guido
type          HR  Temp    HR
year visit                  
2013 1      36.0  35.1  48.0
     2      41.0  38.3  41.0
2014 1      30.0  37.5  31.0

In [19]:
health_data.iloc[:4, :4]

subject      Bob       Guido      
type          HR  Temp    HR  Temp
year visit                        
2013 1      36.0  35.1  48.0  37.9
     2      41.0  38.3  41.0  37.7
2014 1      30.0  37.5  31.0  37.5
     2      32.0  38.6  47.0  36.9

These indexers provide an array-like view of the underlying two-dimensional data, but each individual index in ``loc`` or ``iloc`` can be passed a tuple of multiple indices. For example:

In [20]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        36.0
      2        41.0
2014  1        30.0
      2        32.0
Name: (Bob, HR), dtype: float64

You could get around this by building the desired slice explicitly using Python's built-in ``slice()`` function, but a better way in this context is to use an ``IndexSlice`` object, which Pandas provides for precisely this situation.
For example:

In [45]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,36.0,48.0,39.0
2014,1,30.0,31.0,38.0


In [46]:
idx[:, 1]

(slice(None, None, None), 1)

In [36]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      36.0  35.1  48.0  37.9  39.0  38.0
     2      41.0  38.3  41.0  37.7  39.0  36.0
2014 1      30.0  37.5  31.0  37.5  38.0  35.9
     2      32.0  38.6  47.0  36.9  19.0  37.3

In [42]:
health_data.loc[idx[2013, :], idx[:, 'Temp']]

subject      Bob Guido   Sue
type        Temp  Temp  Temp
year visit                  
2013 1      35.1  37.9  38.0
     2      38.3  37.7  36.0

In [43]:
health_data.index.names

FrozenList(['year', 'visit'])

### Sorted and unsorted indices

Earlier, we briefly mentioned a caveat, but we should emphasize it more here.
*Many of the ``MultiIndex`` slicing operations will fail if the index is not sorted.*
Let's take a look at this here.

We'll start by creating some simple multiply indexed data where the indices are *not lexographically sorted*:

In [22]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.489977
      2      0.461907
c     1      0.138935
      2      0.767014
b     1      0.077798
      2      0.676006
dtype: float64

In [23]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Although it is not entirely clear from the error message, this is the result of the MultiIndex not being sorted.
For various reasons, partial slices and other similar operations require the levels in the ``MultiIndex`` to be in sorted (i.e., lexographical) order.
Pandas provides a number of convenience routines to perform this type of sorting; examples are the ``sort_index()`` and ``sortlevel()`` methods of the ``DataFrame``.
We'll use the simplest, ``sort_index()``, here:

In [24]:
data = data.sort_index()
data

char  int
a     1      0.489977
      2      0.461907
b     1      0.077798
      2      0.676006
c     1      0.138935
      2      0.767014
dtype: float64

In [25]:
data['a':'b']

char  int
a     1      0.489977
      2      0.461907
b     1      0.077798
      2      0.676006
dtype: float64

### Index setting and resetting

Another way to rearrange hierarchical data is to turn the index labels into columns; this can be accomplished with the ``reset_index`` method.
Calling this on the population dictionary will result in a ``DataFrame`` with a *state* and *year* column holding the information that was formerly in the index.
For clarity, we can optionally specify the name of the data for the column representation:

In [26]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


Often when working with data in the real world, the raw input data looks like this and it's useful to build a ``MultiIndex`` from the column values.
This can be done with the ``set_index`` method of the ``DataFrame``, which returns a multiply indexed ``DataFrame``:

In [27]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## Data Aggregations on Multi-Indices

We've previously seen that Pandas has built-in data aggregation methods, such as ``mean()``, ``sum()``, and ``max()``.
For hierarchically indexed data, these can be passed a ``level`` parameter that controls which subset of the data the aggregate is computed on.

For example, let's return to our health data:

In [28]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      36.0  35.1  48.0  37.9  39.0  38.0
     2      41.0  38.3  41.0  37.7  39.0  36.0
2014 1      30.0  37.5  31.0  37.5  38.0  35.9
     2      32.0  38.6  47.0  36.9  19.0  37.3

Perhaps we'd like to average-out the measurements in the two visits each year. We can do this by naming the index level we'd like to explore, in this case the year:

In [29]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido         Sue      
type       HR   Temp    HR  Temp    HR  Temp
year                                        
2013     38.5  36.70  44.5  37.8  39.0  37.0
2014     31.0  38.05  39.0  37.2  28.5  36.6

By further making use of the ``axis`` keyword, we can take the mean among levels on the columns as well:

In [30]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,40.666667,37.166667
2014,32.833333,37.283333


## Aggregation and transformation

### Aggregation

In [31]:
df = pd.DataFrame(np.random.rand(4,4), columns=list('abcd'))
df['group'] = [0, 0, 1, 1]
df

,a,b,c,d,group
0,0.790604,0.925708,0.663606,0.835403,0
1,0.473950,0.531049,0.200434,0.417424,0
2,0.968729,0.050718,0.188612,0.442266,1
3,0.836406,0.371129,0.302005,0.544485,1


Aggregation by means of a dictionary mapped from column names

In [32]:
df.groupby('group').agg({'a':['sum', 'max'], 
                         'b':'mean', 
                         'c':'sum', 
                         'd': lambda x: x.max() - x.min()})


a                   b         c         d
            sum       max      mean       sum  <lambda>
group                                                  
0      1.264555  0.790604  0.728378  0.864040  0.417979
1      1.805135  0.968729  0.210923  0.490617  0.102219

If you don't like that ugly lambda column name, you can use a normal function and supply a custom name to the special __name__ attribute like this:

In [33]:
def max_min(x):
    return x.max() - x.min()

max_min.__name__ = 'Max minus Min'

df.groupby('group').agg({'a':['sum', 'max'], 
                         'b':'mean', 
                         'c':'sum', 
                         'd': max_min})

a                   b         c             d
            sum       max      mean       sum Max minus Min
group                                                      
0      1.264555  0.790604  0.728378  0.864040      0.417979
1      1.805135  0.968729  0.210923  0.490617      0.102219

### Using apply and returning a Series

Now, if you had multiple columns that needed to interact together then you cannot use agg, which implicitly passes a Series to the aggregating function. When using apply the entire group as a DataFrame gets passed into the function.

A possibility is making a single custom function that returns a Series of all the aggregations. Use the Series index as labels for the new columns:

In [34]:
def f(x):
    d = {}
    d['a_sum'] = x['a'].sum()
    d['a_max'] = x['a'].max()
    d['b_mean'] = x['b'].mean()
    d['c_d_prodsum'] = (x['c'] * x['d']).sum()
    return pd.Series(d, index=['a_sum', 'a_max', 'b_mean', 'c_d_prodsum'])

df.groupby('group').apply(f)


,a_sum,a_max,b_mean,c_d_prodsum
group,,,,
0,1.264555,0.790604,0.728378,0.638044
1,1.805135,0.968729,0.210923,0.247854
